Ground resolution
Ordered Ground Resolution

Ground paramodulation
Ground superposition

https://events.model.in.tum.de/mod23/lectures.html Jasmin Blanchettes lectures
https://matryoshka-project.github.io/pubs/satur_report.pdf A Comprehensive Framework for
Saturation Theorem Proving




In [ ]:
def ground_size(t):
        match t:
            case (t1,args):
                return 1 + sum(map, ground_size, args)
def ground_kbo(t1,t2):
    s1 = ground_size(t1)
    s2 = ground_size(t2)
    if s1 < s2:
        return True
    if s1 > s2:
        return False
    elif s1 == s2:
        if t1[0] < t2[0]:
            return True
        elif t1[0] > t2[0]:
            return False
        elif t1[0] == t2[0]:
            for i in range(1,len(t1)):
                if t1[i] < t2[i]:
                    return True
                elif t1[i] > t2[i]:
                    return False
            return False
    
egraph = []




In [1]:
from dataclasses import dataclass
from typing import Any

@dataclass(frozen=True)
class Clause():
    hyps: frozenset[Any]
    concs: frozenset[Any]

def maxlit(c : Clause):
    return max(max(c.concs), max(c.hyps))


Some ideas about  hash consing.
Could store more data in the nodes.
numpy it or flatten it
range queries with stdlib bisect
We could uniquely label vars by depth first ordering. But then if they go in a frozenset, this becomes confusing.


In [14]:
from dataclasses import dataclass
from typing import Any
@dataclass(frozen=True)
class Term():
    pass
@dataclass(frozen=True)
class Fn(Term):
    name: str
    args: tuple[Any, ...] = ()
    def __repr__(self):
        if len(self.args) == 0:
            return self.name
        return f"{self.name}({', '.join(map(repr, self.args))})"
    def __eq__(self, other, perm=[]):
        if not isinstance(other, Fn) or self.name != other.name or len(self.args) != len(other.args):
            return False
        for x,y in zip(self.args, other.args):
            if not x.__eq__(y,perm):
                return False
        return True

    
@dataclass(frozen=True)
class Var(Term):
    name: str
    def __repr__(self):
        return "?" + self.name
    def __eq__(self, other, perm=[]):
        if not isinstance(other, Var):
            return False
        for x,y in perm:
            if x == self.name:
                return y == other.name
            if y == other.name:
                return x == self.name
        perm.append((self.name, other.name))
        return True
    def __hash__(self):
        # The name is not alpha invariant so can't naively be part of the hash
        return 19

def f(x):
    return Fn("f", (x,))
def bar(x,y):
    return Fn("bar", (x,y))
x,y,z = map(Var, "xyz")

print(f"{f(x) == f(f(x))=}")
print(f"{f(x) == f(y)=}")
print(f"{bar(x,y) == bar(x,x)=}")
print(f"{bar(x,x) == bar(y,y)=}")

print(f"{hash(bar(x,y))=} {hash(bar(x,y))=}")

print(f"{set([ bar(x,x), bar(y,y), bar(y,x), bar(x,y) ])=}")


f(x) == f(f(x))=False
f(x) == f(y)=True
bar(x,y) == bar(x,x)=False
bar(x,x) == bar(y,y)=True
hash(bar(x,y))=-2698517946957590065 hash(bar(x,y))=-2698517946957590065
set([ bar(x,x), bar(y,y), bar(y,x), bar(x,y) ])={bar(?y, ?x), bar(?x, ?x)}


In [50]:
import lark
from lark import v_args
@dataclass(frozen=True)
class Fact():
    head: Fn
    def __repr__(self):
        return f"{self.head}."
@dataclass(frozen=True)
class Rule():
    head: Fn
    body: tuple[Fn, ...]
    def __repr__(self):
        return f"{self.head} :- {', '.join(map(repr, self.body))}."

grammar = """
?start : decl* 
?decl :  fact | rule
?fact : term "."
?rule : term ":-" term ("," term)* "."
?term : NAME "(" term ("," term)* ")" -> fn
      | "?" NAME -> var
      | NAME -> const
%import common.CNAME -> NAME
%import common.WS
%ignore WS
"""
class Transformer(lark.Transformer):
    fn = v_args(inline=True)(lambda self, n, *a: Fn(n, a))
    const = v_args(inline=True)(Fn)
    var = v_args(inline=True)(Var)
    fact = v_args(inline=True)(Fact)
    rule = v_args(inline=True)(lambda self, head, *body: Rule(head, body))
    start = list

datalog_parser = lark.Lark(grammar, parser="lalr", transformer=Transformer())

datalog_parser.parse("f(x). f(x) :- bar, biz(?X).")[1]

f(x) :- bar, biz(?X).

In [ ]:
# nominal?
@dataclass(frozen=True)
class Atom():
    name:str

In [51]:
import bisect

class SortedListWithLE:
    def __init__(self):
        self._list = []

    def insert(self, value):
        bisect.insort_left(self._list, value)

    def search_le(self, value):
        index = bisect.bisect_right(self._list, value)
        if index:
            return self._list[index - 1]
        else:
            return None

# Example usage:
sl = SortedListWithLE()
sl.insert(5)
sl.insert(3)
sl.insert(7)
sl.insert(1)
sl.insert(4)
sl.insert(6)
sl.insert(9)

print(sl.search_le(2))  # Output: 1
print(sl.search_le(5))  # Output: 5
print(sl.search_le(8))  # Output: 7


1
5
7


Prolog has all the bits and pieces. WHy can't I just use prolog as my loigcal framework?
Prolog is a pretty powerful metalanguage. Bindings are going to stink.
Amy Felty. Could also try using elpi





In [59]:
# https://www.swi-prolog.org/pldoc/doc_for?object=section(%27packages/janus.html%27)
import janus_swi as janus
#janus.consult("/tmp/trs.pl")
janus.consult("path", """
edge(a,b).
edge(b,c).    
edge(c,d).

:- table path/2.
path(X,Y) :- edge(X,Y).
path(X,Y) :- edge(X,Z), path(Z,Y).
""")
#ans = janus.query_once("normal_form([f(x) ==> g(x)], f(x), _Res), term_to_list(_Res,Res).")
#ans['Res']
list(janus.query("path(a,Y)."))

[{'truth': True, 'Y': 'c'},
 {'truth': True, 'Y': 'b'},
 {'truth': True, 'Y': 'd'}]

In [ ]:
janus.consult("sequent", """\
% unify with occurs check
theorem(refl, [], A = A, assume). % name, hyps, concs, reason
              
cnf(refl,axiom,[A = A]). % this basically is what tptp _is_

% could reify stuff too. The sequent becomes a clause.
% A == A.            
              
% modus takes in the _names_ of two theorems and returns a new theorem.
modus(a,b,X) :- var(X), theorem(a, S, C, _), theorem(b, C, D, _), gensym(X), asserta(theorem(X, S, D, modus(a,b))). 
              % ammusingly call on the reason would basically prove the thing again.
reflect(C) :- C, gensym(X), asserta(theorem(X,[],C)).

""")